<a href="https://colab.research.google.com/github/tabaahi07/Research_Paper/blob/main/Research_paper_Optimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt

from keras.datasets import mnist
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

In [4]:
# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [5]:
# Normalize pixel values to range [0, 1]
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, 28 * 28))
X_test_scaled = scaler.transform(X_test.reshape(-1, 28 * 28))

In [ ]:
# Hyperparameter Tuning for SVM
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
svm = GridSearchCV(SVC(), param_grid, refit=True, verbose=2)
svm.fit(X_train_scaled, y_train)
y_pred_svm = svm.predict(X_test_scaled)
print("SVM Accuracy: ", accuracy_score(y_test, y_pred_svm))


Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] END ...............................C=0.1, kernel=linear; total time= 2.9min
[CV] END ...............................C=0.1, kernel=linear; total time= 2.9min
[CV] END ...............................C=0.1, kernel=linear; total time= 2.9min
[CV] END ...............................C=0.1, kernel=linear; total time= 2.8min
[CV] END ...............................C=0.1, kernel=linear; total time= 2.9min
[CV] END ..................................C=0.1, kernel=rbf; total time= 9.2min


In [1]:
# Random Forest with PCA for dimensionality reduction
pca = PCA(n_components=100)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_pca, y_train)
y_pred_rf = rf.predict(X_test_pca)
print("Random Forest Accuracy: ", accuracy_score(y_test, y_pred_rf))

NameError: name 'PCA' is not defined

In [ ]:
# Transfer Learning using VGG16 for CNN
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(28, 28, 1))

model = Sequential()
model.add(vgg)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train.reshape(-1, 28, 28, 1), y_train, epochs=5, validation_data=(X_test.reshape(-1, 28, 28, 1), y_test))

y_pred_cnn = np.argmax(model.predict(X_test.reshape(-1, 28, 28, 1)), axis=-1)
print("CNN with Transfer Learning Accuracy: ", accuracy_score(y_test, y_pred_cnn))

In [ ]:
# Evaluate all models
results = {
    'SVM': {
        'accuracy': accuracy_score(y_test, y_pred_svm),
        'f1_score': f1_score(y_test, y_pred_svm, average='weighted'),
        'precision': precision_score(y_test, y_pred_svm, average='weighted'),
        'recall': recall_score(y_test, y_pred_svm, average='weighted')
    },
    'Random Forest': {
        'accuracy': accuracy_score(y_test, y_pred_rf),
        'f1_score': f1_score(y_test, y_pred_rf, average='weighted'),
        'precision': precision_score(y_test, y_pred_rf, average='weighted'),
        'recall': recall_score(y_test, y_pred_rf, average='weighted')
    },
    'CNN with Transfer Learning': {
        'accuracy': accuracy_score(y_test, y_pred_cnn),
        'f1_score': f1_score(y_test, y_pred_cnn, average='weighted'),
        'precision': precision_score(y_test, y_pred_cnn, average='weighted'),
        'recall': recall_score(y_test, y_pred_cnn, average='weighted')
    }
}


In [ ]:
# Display results
import tabulate
table = [[model, metrics['accuracy'], metrics['f1_score'], metrics['precision'], metrics['recall']] for model, metrics in results.items()]
print(tabulate.tabulate(table, headers=['Model', 'Accuracy', 'F1 Score', 'Precision', 'Recall'], tablefmt='grid'))